In [10]:
import pandas as pd
import time
from glob import glob
import os

import ccxt
from data_provider import read_data_path, market_to_symbol

hold = 30
exchange = ccxt.binance()

data_path = read_data_path()
cash_data_path = data_path + '/Cash'

if glob(cash_data_path) == []:
    os.makedirs(cash_data_path)

In [3]:
cash_data_path

'/home/user/Robots/Data/Binance/Cash'

In [ ]:
last_time = 0
run = True
while run:
    now = exchange.milliseconds()
    df = pd.DataFrame({'timestamp':[now], 'time':[exchange.iso8601(now)]})
    file_name = 'Logs/cash_reader.state'
    df.to_csv(file_name, index = False)

    df = pd.read_csv("markets_in_trade.csv")
    for row in df.iterrows():
        time.sleep (exchange.rateLimit * 2 / 1000) # time.sleep wants seconds

        market = row[1]['market']
        tf = row[1]['tf']
        symbol = market_to_symbol(market)
        try:
            ohlcvs =  exchange.fetch_ohlcv (symbol, tf)
        except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:
            #error_log = exchange.iso8601(now), 'Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...'
            error_log = str(exchange.iso8601(now)), 'Got an error', type(error).__name__, ', retrying in', str(hold), 'seconds...'
            error_log = ' '.join(error_log)
            with open(cash_data_path +'/errors.log', 'a') as myfile:
                myfile.write(error_log)
            time.sleep(hold)  
      
   
        df = pd.DataFrame(ohlcvs)
        df.columns = ['T', 'O', 'H', 'L', 'C', 'V']
        file_name = cash_data_path + '/' + market + '_' + tf + '.csv'
        df.to_csv(file_name, index = False)                          
        
        #if last_time < df.iloc[-1]['T']:
        #    df.to_csv(file_name, index = False)                  
        #    last_time = df.iloc[-1]['T']